In [1]:
pip install pydicom

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pydicom
import pandas as pd
from glob import glob
import os
from matplotlib.patches import Rectangle
import seaborn as sns
import cv2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import keras
from keras.applications.densenet import DenseNet121
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
#from generator import DataGenerator

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Convolution2D, Dropout, Dense,Conv2D
from keras.layers import BatchNormalization,Activation
from keras.layers import MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Flatten
from keras.optimizers import adam
from keras import optimizers


In [6]:
cd drive/My Drive/Colab Notebooks/RSNA/rsna-pneumonia-detection-challenge

/content/drive/My Drive/Colab Notebooks/RSNA/rsna-pneumonia-detection-challenge


In [7]:
!ls

 brucechou1983_CheXNet_Keras_0.3.0_weights.h5   train_png
'GCP Credits Request Link - RSNA.txt'	        transfer_all_sgd1.h5
 stage_2_detailed_class_info.csv	        transfer_all_sgd2.h5
 stage_2_sample_submission.csv		        transfer_all_sgd3000.h5
 stage_2_test_images			        transfer_all_sgd.h5
 stage_2_train_images			        validation_png
 stage_2_train_labels.csv


In [14]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        #horizontal_flip=True
         )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator_1 = train_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/RSNA/rsna-pneumonia-detection-challenge/train_png/',
        target_size=(256, 256),
        batch_size=32,color_mode='rgb',
        class_mode='binary')

validation_generator_1 = test_datagen.flow_from_directory(
        '/content/drive/My Drive/Colab Notebooks/RSNA/rsna-pneumonia-detection-challenge/validation_png/',
        target_size=(256, 256),
        batch_size=32,color_mode='rgb',
        class_mode='binary')

Found 4838 images belonging to 2 classes.
Found 441 images belonging to 2 classes.


In [0]:
# input_shape=(256,256,3)
# img_in = Input(input_shape)              #input of model 
# model = DenseNet121(include_top= False , # remove  the 3 fully-connected layers at the top of the network
#                 weights='/content/drive/My Drive/Colab Notebooks/RSNA/rsna-pneumonia-detection-challenge/brucechou1983_CheXNet_Keras_0.3.0_weights.h5',      # pre train weight 
#                 input_tensor= img_in, 
#                 input_shape= input_shape,
#                 pooling ='max') 



In [0]:
input_shape=(256,256,3)
img_in = Input(input_shape)              #input of model 
model = DenseNet121(include_top= False , # remove  the 3 fully-connected layers at the top of the network
                weights='imagenet',      # pre train weight 
                input_tensor= img_in, 
                input_shape= input_shape,
                pooling ='max') 

In [39]:
x = model.get_output_at(-1)
#layer_output = self.layer.get_output_at(-1)
predictions = Dense(1, activation="sigmoid",name="predictions")(x)
model11 = Model(inputs=img_in, outputs=predictions)

RuntimeError: ignored

In [34]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/RSNA/rsna-pneumonia-detection-challenge/brucechou1983_CheXNet_Keras_0.3.0_weights.h5')

ValueError: ignored

In [9]:
# Create the model
model_pretrainedW_3000= Sequential()
 
# Add the vgg convolutional base model
model_pretrainedW_3000.add(model)
 
# Add new layers
#model_vgg_flow.add(Flatten())
#model_Densenet.add(Dense(256, activation='relu'))
#model_Densenet.add(Dropout(0.25))
#model_Densenet.add(Dense(128, activation='relu'))
model_pretrainedW_3000.add(Dropout(0.25))
model_pretrainedW_3000.add(Dense(256, activation='relu'))
model_pretrainedW_3000.add(Dropout(0.25))
model_pretrainedW_3000.add(Dense(128, activation='relu'))
model_pretrainedW_3000.add(Dropout(0.15))
model_pretrainedW_3000.add(Dense(64, activation='relu'))
model_pretrainedW_3000.add(Dense(1, activation='sigmoid'))

#model_vgg_flow.add(Dense(1, activation='softmax')) 
# Show a summary of the model. Check the number of trainable parameters
model_pretrainedW_3000.summary()

W0625 04:22:01.039360 140541277095808 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1024)              7037504   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
__________

In [15]:
batch_size=128
sgd = optimizers.SGD(lr=0.03, decay=1e-6, momentum=0.9, nesterov=True)
model_pretrainedW_3000.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])
model_pretrainedW_3000.fit_generator(
        train_generator_1,
        steps_per_epoch=4838 // batch_size,
        epochs=3,
        validation_data=validation_generator_1,
        validation_steps=441// 2)
model_pretrainedW_3000.save_weights('/content/drive/My Drive/Colab Notebooks/RSNA/rsna-pneumonia-detection-challenge/transfer_all_sgd3000.h5')  # always save your weights after training or during training


Epoch 1/3
37/37 [==============================] - 322s 9s/step - loss: 0.8706 - acc: 0.6157 - val_loss: 0.7145 - val_acc: 0.5687
Epoch 2/3
37/37 [==============================] - 262s 7s/step - loss: 0.6391 - acc: 0.6647 - val_loss: 0.5604 - val_acc: 0.7346
Epoch 3/3
37/37 [==============================] - 258s 7s/step - loss: 0.5470 - acc: 0.7424 - val_loss: 0.7126 - val_acc: 0.6749
